In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('C:/Users/ravee/Jupyter/Skin-Cancer-Classification/data/glcm3.csv', header=None)
X = dataset.iloc[:, 0:24].values
Y = dataset.iloc[:, 24].values

In [7]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.111339,0.152491,0.114248,0.153011,0.109154,0.147662,0.112359,0.148381,0.945635,0.926633,...,0.265834,0.995359,0.993634,0.995237,0.993613,0.076647,0.070833,0.076181,0.070668,1
1,0.118241,0.158798,0.123516,0.164937,0.118169,0.158605,0.123435,0.164563,0.940922,0.920717,...,0.273114,0.992255,0.989582,0.991906,0.989182,0.082170,0.075512,0.081259,0.074591,1
2,0.096371,0.152091,0.117267,0.141002,0.095205,0.145822,0.114591,0.138458,0.952510,0.927680,...,0.301113,0.994811,0.991801,0.993681,0.992398,0.099094,0.089396,0.095389,0.090669,1
3,0.088059,0.119696,0.088056,0.115000,0.088009,0.119331,0.087956,0.114751,0.956000,0.940371,...,0.423064,0.987305,0.982757,0.987311,0.983433,0.187850,0.178278,0.187297,0.178983,1
4,0.183870,0.254857,0.172721,0.237534,0.175467,0.231480,0.166016,0.220323,0.913104,0.886569,...,0.304045,0.977174,0.968355,0.978547,0.970506,0.100605,0.090623,0.102176,0.092443,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,0.136390,0.188860,0.145767,0.186687,0.136127,0.187470,0.145519,0.185561,0.931963,0.906404,...,0.307004,0.986611,0.981444,0.985685,0.981658,0.103888,0.093887,0.101714,0.094251,3
917,0.204086,0.261437,0.198129,0.275890,0.201532,0.253912,0.196466,0.268425,0.899489,0.873792,...,0.259094,0.985333,0.981057,0.985738,0.980027,0.075938,0.068317,0.076629,0.067130,3
918,0.172552,0.215125,0.161815,0.229059,0.171038,0.211278,0.160423,0.224471,0.914632,0.894745,...,0.240843,0.990387,0.987980,0.990964,0.987218,0.064741,0.059560,0.066223,0.058005,3
919,0.254489,0.335740,0.268282,0.366426,0.242730,0.305959,0.249562,0.326843,0.879803,0.849980,...,0.216076,0.987113,0.982953,0.986366,0.981405,0.054408,0.048588,0.053877,0.046689,3


In [3]:
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [4]:
len(X_train)

736

In [5]:
len(X_test)

185

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Naive Bayes

In [7]:
# Implementation of Naive Bayes
# Fitting Naive Bayes to the training set

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [8]:
# Predicting the test set result
y_pred = nb.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[62, 14,  3],
       [30, 23,  7],
       [28, 14,  4]], dtype=int64)

In [9]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = nb, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.5217697149203998


In [10]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = nb, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.52      0.78      0.62        40
           2       0.33      0.22      0.26        32
           3       0.50      0.29      0.36        21

    accuracy                           0.47        93
   macro avg       0.45      0.43      0.42        93
weighted avg       0.45      0.47      0.44        93

              precision    recall  f1-score   support

           1       0.58      0.87      0.69        39
           2       0.72      0.64      0.68        33
           3       0.50      0.10      0.17        20

    accuracy                           0.62        92
   macro avg       0.60      0.54      0.51        92
weighted avg       0.61      0.62      0.57        92

              precision    recall  f1-score   support

           1       0.54      0.82      0.65        39
           2       0.61      0.42      0.50        33
           3       0.40      0.20      0.27        20

    accuracy        

In [11]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.53      0.79      0.63       391
         bcc       0.54      0.43      0.48       323
         scc       0.33      0.13      0.19       207

    accuracy                           0.51       921
   macro avg       0.47      0.45      0.43       921
weighted avg       0.49      0.51      0.48       921



# K-Nearest Neighbor

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 25, metric = 'minkowski', p = 1, weights = 'distance')
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=25, p=1,
                     weights='distance')

In [13]:
# Predicting the test set result
y_pred = knn.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[56, 17,  6],
       [23, 32,  5],
       [18, 20,  8]], dtype=int64)

In [14]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = knn, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.578822658274713


In [15]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'n_neighbors': [21, 23, 25, 27, 29], 'weights': ['uniform', 'distance'], 'p': [1, 2, 3], 'metric': ['minkowski']},
]

grid_search = GridSearchCV(estimator = knn, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.58
{'metric': 'minkowski', 'n_neighbors': 25, 'p': 1, 'weights': 'distance'}


In [16]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = knn, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.60      0.75      0.67        40
           2       0.46      0.50      0.48        32
           3       0.62      0.24      0.34        21

    accuracy                           0.55        93
   macro avg       0.56      0.50      0.50        93
weighted avg       0.56      0.55      0.53        93

              precision    recall  f1-score   support

           1       0.64      0.69      0.67        39
           2       0.59      0.79      0.68        33
           3       0.50      0.15      0.23        20

    accuracy                           0.61        92
   macro avg       0.58      0.54      0.52        92
weighted avg       0.59      0.61      0.58        92

              precision    recall  f1-score   support

           1       0.65      0.72      0.68        39
           2       0.52      0.67      0.59        33
           3       0.71      0.25      0.37        20

    accuracy        

In [17]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.61      0.72      0.66       391
         bcc       0.53      0.62      0.57       323
         scc       0.51      0.22      0.31       207

    accuracy                           0.57       921
   macro avg       0.55      0.52      0.51       921
weighted avg       0.56      0.57      0.55       921



# Support Vector Machine

In [18]:
# Implementation of Support Vector Machine (SVM)
# Fitting SVM to the training set

from sklearn.svm import SVC
svm = SVC(C = 100, kernel = 'rbf', gamma = 0.01, random_state = 0)
#classifier = SVC(C = 1, kernel = 'poly', degree = 30, random_state = 0)
svm.fit(X_train, y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [19]:
# Predicting the test set result
y_pred = svm.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[58, 21,  0],
       [21, 38,  1],
       [18, 27,  1]], dtype=int64)

In [20]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = svm, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.5856349500185116


In [10]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'C': [1, 10, 100], 'kernel': ['rbf', 'sigmoid'], 'gamma': [0.001, 0.01, 0.1]},
]

grid_search = GridSearchCV(estimator = svm, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.59
{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}


In [21]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred, zero_division = 0))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = svm, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.49      0.82      0.61        40
           2       0.40      0.31      0.35        32
           3       0.00      0.00      0.00        21

    accuracy                           0.46        93
   macro avg       0.30      0.38      0.32        93
weighted avg       0.35      0.46      0.38        93

              precision    recall  f1-score   support

           1       0.59      0.90      0.71        39
           2       0.73      0.73      0.73        33
           3       0.00      0.00      0.00        20

    accuracy                           0.64        92
   macro avg       0.44      0.54      0.48        92
weighted avg       0.51      0.64      0.56        92

              precision    recall  f1-score   support

           1       0.51      0.87      0.64        39
           2       0.52      0.39      0.45        33
           3       0.00      0.00      0.00        20

    accuracy        

In [22]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names, zero_division = 0))

              precision    recall  f1-score   support

    melanoma       0.52      0.84      0.64       391
         bcc       0.55      0.48      0.51       323
         scc       0.00      0.00      0.00       207

    accuracy                           0.53       921
   macro avg       0.35      0.44      0.38       921
weighted avg       0.41      0.53      0.45       921



# Decision Tree

In [23]:
# Some important notes here, before running the Decision Tree Classifier, it's ok to not scale the features, as this model
# does not work using euclidian distance, so it's fine as it is

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10, random_state = 0, splitter = 'random')
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='random')

In [24]:
# Predicting the test set result
y_pred = tree.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[43, 26, 10],
       [15, 33, 12],
       [18, 20,  8]], dtype=int64)

In [25]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = tree, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.5287671232876712


In [36]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'criterion': ['entropy', 'gini'], 'splitter': ['best', 'random'], 'ccp_alpha': [0.0, 0.05, 0.1], 'max_depth': [10, 100, 200]}
]

grid_search = GridSearchCV(estimator = tree, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.53
{'ccp_alpha': 0.05, 'criterion': 'entropy', 'max_depth': 10, 'splitter': 'random'}


In [26]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = tree, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.58      0.72      0.64        40
           2       0.46      0.41      0.43        32
           3       0.27      0.19      0.22        21

    accuracy                           0.49        93
   macro avg       0.44      0.44      0.43        93
weighted avg       0.47      0.49      0.48        93

              precision    recall  f1-score   support

           1       0.57      0.54      0.55        39
           2       0.57      0.61      0.59        33
           3       0.30      0.30      0.30        20

    accuracy                           0.51        92
   macro avg       0.48      0.48      0.48        92
weighted avg       0.51      0.51      0.51        92

              precision    recall  f1-score   support

           1       0.51      0.54      0.53        39
           2       0.44      0.58      0.50        33
           3       0.38      0.15      0.21        20

    accuracy        

In [27]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.57      0.58      0.58       391
         bcc       0.45      0.53      0.49       323
         scc       0.24      0.17      0.20       207

    accuracy                           0.47       921
   macro avg       0.42      0.43      0.42       921
weighted avg       0.46      0.47      0.46       921



# Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 50, max_depth = None, criterion = 'entropy', class_weight = 'balanced_subsample', random_state = 0)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=50, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [29]:
# Predicting the test set result
y_pred = forest.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[57, 17,  5],
       [20, 33,  7],
       [20, 17,  9]], dtype=int64)

In [30]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = forest, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(np.mean(accuracies))

0.5584228063680119


In [41]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'criterion': ['entropy'], 'max_depth': [None, 10], 'n_estimators': [50, 100, 150], 'class_weight': [None, 'balanced', 'balanced_subsample']},
    {'criterion': ['gini'], 'max_depth': [None, 10], 'n_estimators': [50, 100, 150], 'class_weight': [None, 'balanced', 'balanced_subsample']}
]

grid_search = GridSearchCV(estimator = forest, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.57
{'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'n_estimators': 50}


In [31]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = forest, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.50      0.55      0.52        40
           2       0.30      0.31      0.31        32
           3       0.25      0.19      0.22        21

    accuracy                           0.39        93
   macro avg       0.35      0.35      0.35        93
weighted avg       0.38      0.39      0.38        93

              precision    recall  f1-score   support

           1       0.64      0.69      0.67        39
           2       0.68      0.76      0.71        33
           3       0.46      0.30      0.36        20

    accuracy                           0.63        92
   macro avg       0.59      0.58      0.58        92
weighted avg       0.62      0.63      0.62        92

              precision    recall  f1-score   support

           1       0.73      0.56      0.64        39
           2       0.50      0.64      0.56        33
           3       0.50      0.50      0.50        20

    accuracy        

In [32]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.61      0.66      0.63       391
         bcc       0.53      0.57      0.55       323
         scc       0.42      0.29      0.34       207

    accuracy                           0.55       921
   macro avg       0.52      0.51      0.51       921
weighted avg       0.54      0.55      0.54       921



# Artificial Neural Network

In [38]:
# Importing the Keras libraries and packages
import keras
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [39]:
# Create categorical input for predictor

from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore')
Y = Y.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
y_use = onehotencoder.fit_transform(Y).toarray()
y_binary = onehotencoder.fit_transform(y_train).toarray()
y_compare = onehotencoder.fit_transform(y_test).toarray()

In [40]:
# Initializing the ANN
ann = Sequential()

# Adding the input & hidden layer
ann.add(Dense(32, activation = 'relu', input_shape = (24,)))
ann.add(Dense(32, activation = 'relu'))
ann.add(Dense(32, activation = 'relu'))
ann.add(Dense(3, activation = 'softmax'))
ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ann.fit(X_train, y_binary, batch_size = 10, epochs = 100)

Epoch 1/100
736/736 [==============================] - 0s 214us/step - loss: 1.0257 - accuracy: 0.5054
Epoch 2/100
736/736 [==============================] - 0s 95us/step - loss: 0.9471 - accuracy: 0.5516
Epoch 3/100
736/736 [==============================] - 0s 95us/step - loss: 0.9274 - accuracy: 0.5625
Epoch 4/100
736/736 [==============================] - 0s 91us/step - loss: 0.9266 - accuracy: 0.5543
Epoch 5/100
736/736 [==============================] - 0s 89us/step - loss: 0.9149 - accuracy: 0.5611
Epoch 6/100
736/736 [==============================] - 0s 92us/step - loss: 0.9088 - accuracy: 0.5693
Epoch 7/100
736/736 [==============================] - 0s 99us/step - loss: 0.9020 - accuracy: 0.5652
Epoch 8/100
736/736 [==============================] - 0s 101us/step - loss: 0.9029 - accuracy: 0.5747
Epoch 9/100
736/736 [==============================] - 0s 98us/step - loss: 0.8955 - accuracy: 0.5747
Epoch 10/100
736/736 [==============================] - 0s 91us/step - loss: 0.8

In [41]:
# Create threshold for multi-predictor
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [42]:
# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_compare.argmax(axis=1), y_pred.argmax(axis=1))

cm

array([[62, 15,  2],
       [30, 27,  3],
       [24, 13,  9]], dtype=int64)

In [44]:
# Calculating the accuracy and other metrics
#, scoring=make_scorer(classification_report_with_accuracy_score)

from sklearn.model_selection import StratifiedKFold, cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Build the Model
def buildmodel():
    ann = Sequential()
    ann.add(Dense(16, activation = 'relu', input_shape = (24,)))
    ann.add(Dense(16, activation = 'relu'))
    ann.add(Dense(3, activation = 'softmax'))
    ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_accuracy'])
    return(ann)


#Make our customer score
def classification_report_with_accuracy_score(y_use, y_pred):
    originalclass.extend(y_use)
    predictedclass.extend(y_pred.argmax(axis=1))
    print(classification_report(y_use, y_pred.argmax(axis=1)))
    return accuracy_score(y_use, y_pred.argmax(axis=1)) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
ann = KerasRegressor(build_fn=buildmodel, epochs = 100, batch_size = 10, verbose = 0)
nested_score = cross_val_score(estimator = ann, X = X, y = y_use.argmax(1), cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.54      0.75      0.63        40
           1       0.35      0.41      0.38        32
           2       0.00      0.00      0.00        21

    accuracy                           0.46        93
   macro avg       0.30      0.39      0.33        93
weighted avg       0.35      0.46      0.40        93

              precision    recall  f1-score   support

           0       0.67      0.82      0.74        39
           1       0.64      0.82      0.72        33
           2       0.50      0.05      0.09        20

    accuracy                           0.65        92
   macro avg       0.60      0.56      0.52        92
weighted avg       0.62      0.65      0.59        92



C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.54      0.87      0.67        39
           1       0.55      0.48      0.52        33
           2       0.00      0.00      0.00        20

    accuracy                           0.54        92
   macro avg       0.36      0.45      0.39        92
weighted avg       0.43      0.54      0.47        92

              precision    recall  f1-score   support

           0       0.54      0.90      0.67        39
           1       0.60      0.45      0.52        33
           2       0.00      0.00      0.00        20

    accuracy                           0.54        92
   macro avg       0.38      0.45      0.40        92
weighted avg       0.44      0.54      0.47        92



C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.53      0.82      0.65        39
           1       0.53      0.53      0.53        32
           2       0.00      0.00      0.00        21

    accuracy                           0.53        92
   macro avg       0.35      0.45      0.39        92
weighted avg       0.41      0.53      0.46        92



C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.60      0.79      0.68        39
           1       0.57      0.72      0.64        32
           2       0.00      0.00      0.00        21

    accuracy                           0.59        92
   macro avg       0.39      0.50      0.44        92
weighted avg       0.45      0.59      0.51        92

              precision    recall  f1-score   support

           0       0.59      0.77      0.67        39
           1       0.51      0.66      0.58        32
           2       0.00      0.00      0.00        21

    accuracy                           0.55        92
   macro avg       0.37      0.48      0.41        92
weighted avg       0.43      0.55      0.48        92



C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.56      0.78      0.65       391
         bcc       0.52      0.61      0.56       323
         scc       0.25      0.00      0.01       207

    accuracy                           0.55       921
   macro avg       0.45      0.46      0.41       921
weighted avg       0.48      0.55      0.48       921



# XGBoost

In [33]:
# Importing the XGB Library and fit it to the data
from xgboost import XGBClassifier
xgb = XGBClassifier(objective = 'multi:softmax', booster = 'dart', n_estimators = 100, rate_drop = 0.2, sample_type = 'uniform', normalize_type = 'tree')
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              normalize_type='tree', nthread=None, objective='multi:softprob',
              random_state=0, rate_drop=0.2, reg_alpha=0, reg_lambda=1,
              sample_type='uniform', scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [34]:
# Predicting the test set result
y_pred = xgb.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[57, 21,  1],
       [21, 37,  2],
       [21, 25,  0]], dtype=int64)

In [35]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(np.mean(accuracies))

0.5475379489078119


In [13]:
# Applying Grid Search to Find the Best Hyperparameter
# {'objective': ['multi:softmax'], 'n_estimators': [25, 50, 100], 'booster': ['gbtree'], 'eta': [0.001, 0.002, 0.004], 'max_depth':[3], 'gamma': [0], 'subsample': [1], 'sampling_method': ['uniform', 'gradient_based']},
# {'objective': ['multi:softmax'], 'n_estimators': [109], 'booster': ['dart'], 'sample_type': ['uniform', 'weighted'], 'normalize_type': ['tree', 'forest'], 'rate_drop': [0, 0.1, 0.2], 'skip_drop': [0, 0.1, 0.2]}
# {'objective': ['multi:softmax'], 'n_estimators': [10, 25, 50], 'booster': ['gblinear'], 'feature_selector': ['greedy', 'cyclic', 'random', 'shuffle', 'thrifty'], 'updater': ['shotgun', 'coord_descent'], 'lambda': [0, 0.5, 1], 'alpha': [0, 0.5, 1]}

from sklearn.model_selection import GridSearchCV
parameters = [
    {'objective': ['multi:softmax'], 'n_estimators': [100, 200, 300], 'booster': ['dart'], 'sample_type': ['uniform', 'weighted'], 'normalize_type': ['tree', 'forest'], 'rate_drop': [0, 0.1, 0.2], 'skip_drop': [0, 0.1, 0.2]}
]

grid_search = GridSearchCV(estimator = xgb, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.57
{'booster': 'dart', 'n_estimators': 100, 'normalize_type': 'tree', 'objective': 'multi:softmax', 'rate_drop': 0.2, 'sample_type': 'uniform', 'skip_drop': 0}


In [36]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = xgb, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.64      0.68      0.66        40
           2       0.39      0.62      0.48        32
           3       0.00      0.00      0.00        21

    accuracy                           0.51        93
   macro avg       0.35      0.43      0.38        93
weighted avg       0.41      0.51      0.45        93

              precision    recall  f1-score   support

           1       0.66      0.69      0.68        39
           2       0.56      0.82      0.67        33
           3       0.67      0.10      0.17        20

    accuracy                           0.61        92
   macro avg       0.63      0.54      0.51        92
weighted avg       0.63      0.61      0.56        92

              precision    recall  f1-score   support

           1       0.57      0.67      0.61        39
           2       0.44      0.61      0.51        33
           3       1.00      0.05      0.10        20

    accuracy        

In [37]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.62      0.69      0.65       391
         bcc       0.49      0.69      0.57       323
         scc       0.35      0.04      0.08       207

    accuracy                           0.55       921
   macro avg       0.48      0.47      0.43       921
weighted avg       0.51      0.55      0.49       921

